In [1]:
import numpy as np
import pandas as pd
import math
from functools import reduce


##TODO add the link of each data with description of each column

###  [Alcohol Per Capita consumption (APC)](http://apps.who.int/gho/data/node.main.A1039?lang=en)

###  [Tobacco Use By Country]()

In [52]:
apc_raw = pd.read_csv('data/raw/APC_2010-2018.csv')
tobac_use_raw = pd.read_csv('data/raw/TobaccoUseByCountry_gte15.csv')
country_data = pd.read_csv('data/raw/CountriesOfTheWorld.csv')
tobac_cost_raw = pd.read_csv('data/raw/Tobacco_cost.csv')
tobac_ad_raw = pd.read_csv('data/raw/BanOnDirectAdvertising.csv')
tobac_tax_raw = pd.read_csv('data/raw/tax-Cigarate/Most-Brand-tax/R_Sp_excise_estimate,R_Ad_val_estimate,R_imp_duty_estimate,R_VAT_estimate,R_Other_estimate,R_total_tax_estimate,R_excise_incr.csv')
ppp_raw = pd.read_csv('data/raw/PPP/DP_LIVE_01122019124828802.csv')

In [131]:
def generic_clean(df_raw, 
                  columns, 
                  names_dict=None,
                  drop_indices=None,
                  apply_func_cols=None,
                  save_path=None):
    '''
    Parameters
    ----------
    df_raw: pandas.DataFrame
        the raw dataframe input
    
    columns: list
        list of columns to keep
    
    names_dict: dict
        dictionary of old_name:new_name to rename column names
        
    drop_indices: list
        list of row indices to be deleted
    
    func_cols: list
        list of (func, cols)
        applies the func to the list of columns in cols
    
    map_func: funtion
        the function to be use for dataframe map 

    save_path: str
        path to store the final df
    
    Returns
    -------
    pandas.DataFrame
            Processed dataframe
    '''
    
    df = df_raw[columns]
    if names_dict != None:
        df = df.rename(columns=names_dict)

    if drop_indices != None:
            df.drop(drop_indices, inplace=True)
    df = df.dropna()
    
    if apply_func_cols != None:
        for apply_func_col in apply_func_cols:            
            func, cols = apply_func_col
            for col in cols:
                df[col] = df[col].apply(func)
        
#         tobac_use['Country'].map(lambda x: x.strip())
        
    df.reset_index(inplace=True, drop=True)
    df.reset_index(inplace=True)
    
    if save_path != None:
        df.to_csv(save_path, index=False)

    return df

In [107]:
country_str_strip = (lambda x: x.strip(), ['Country'])

## Tobac_use cleaning

In [77]:
def split_row(row):
    if isinstance(row, str):
        return float(row.split()[0])
    else:
        return row

In [108]:
tobac_use_cols = list(tobac_use_raw.columns)
tobac_use_names = {" Male": "Male", " Female": "Female"}
use_func_cols = [(split_row, ['Male', 'Female']), country_str_strip]
drop_indices_use = list(tobac_use_raw[tobac_use_raw['Year']>2019].index)

In [109]:
tobac_use = generic_clean(df_raw=tobac_use_raw,
             columns=tobac_use_cols,
             names_dict=tobac_use_names,
             drop_indices=drop_indices_use,
             apply_func_cols=use_func_cols,
             save_path='data/tobac_use_by_country.csv')

In [110]:
tobac_use.head()

,index,Country,Year,Male,Female
0,0,Albania,2015,51.2,7.6
1,1,Albania,2012,52.1,8.2
2,2,Albania,2010,53.1,8.7
3,3,Albania,2005,54.7,10.1
4,4,Albania,2000,56.9,11.6


## Tobacco Cost cleaning

In [92]:
tobac_cost_cols = list(tobac_cost_raw.columns)
tobac_cost_names = {"Premium brand of cigarettes - currency reported": "currency",
                                       "Premium brand of cigarettes - price in currency reported": "price in currency",
                                       "Premium brand of cigarettes - price in international dollars": "price in international $",
                                       "Premium brand of cigarettes - price in US$ at official exchange rates": "price in US$"}

In [111]:
cols_for_drop = list(tobac_cost_names.keys())[1:]
not_avail = tobac_cost_raw[tobac_cost_raw[cols_for_drop].eq('Not available').any(axis=1)]
drop_indices_cost = list(not_avail.index)
cost_func_cols = [country_str_strip]

In [100]:
tobac_cost = generic_clean(df_raw=tobac_cost_raw,
              columns=tobac_cost_cols,
              names_dict=tobac_cost_names,
              drop_indices=drop_indices_cost,
              apply_func_cols=cost_func_cols,
              save_path='data/tobac_cost.csv')

In [101]:
tobac_cost.head()

,index,Country,Year,currency,price in currency,price in international $,price in US$
0,0,Afghanistan,2014,AFN,100.00,2.95,1.73
1,1,Albania,2014,ALL,270.00,5.23,2.60
2,2,Algeria,2014,DZD,150.00,2.53,1.91
3,3,Angola,2014,AOA,200.00,2.18,2.06
4,4,Antigua and Barbuda,2014,XCD,8.00,3.99,2.96


# Tobacco Tax

Data from [Most Sold Brand Tax](http://apps.who.int/gho/data/node.main.1309?lang=en)

We use the **Most sold brand of cigarettes - taxes as a % of price - total tax** column of table.


In [112]:
tax_cols = ['Most sold brand of cigarettes - taxes as a % of price - total tax', 'Country', 'Year']
tax_col_names = {'Most sold brand of cigarettes - taxes as a % of price - total tax': 'Tax'}
tax_func_cols = [country_str_strip]

In [113]:
tobac_tax = generic_clean(df_raw=tobac_tax_raw,
                          columns=tax_cols,
                          names_dict=tax_col_names,
                          save_path='data/tobac_tax.csv')

In [114]:
tobac_tax.head()

,index,Tax,Country,Year
0,0,2.80,Afghanistan,2014
1,1,2.45,Afghanistan,2012
2,2,3.60,Afghanistan,2010
3,3,7.79,Afghanistan,2008
4,4,64.10,Albania,2014


# Purchasing power parity (PPP)

In [156]:
ppp_cols = ['LOCATION', 'TIME', 'Value']
ppp_names_dict = {'LOCATION':'Country', 'TIME':'Year', 'Value': 'PPP'} 
ppp_func_cols = [country_str_strip]

In [157]:
ppp_data = generic_clean(df_raw=ppp_raw, 
                         columns=ppp_cols,
                         names_dict=ppp_names_dict,
                         apply_func_cols=ppp_func_cols,
                         save_path="data/ppp_data.csv")

In [155]:
ppp_data.head()

,index,Country,Year,PPP
0,0,AUS,1960,0.700946
1,1,AUS,1961,0.701324
2,2,AUS,1962,0.687990
3,3,AUS,1963,0.698321
4,4,AUS,1964,0.708346


## APC cleaning

In [148]:
apc_cols = ['Country', 'Beverage Types', '2016', '2015', '2014','2013', '2012', '2011', '2010']
apc_func_cols = [country_str_strip]

In [159]:
apc_data = generic_clean(df_raw=apc_raw,
                         columns=apc_cols,
                         apply_func_cols=apc_func_cols)

In [160]:
beverages = [' Beer', ' Wine', ' Spirits', ' Other alcoholic beverages']
apc_data = apc_data[~apc_data['Beverage Types'].isin(beverages)]
apc_data = apc_data.drop(['Beverage Types'], axis=1)

In [161]:
apc_data.head()

,index,Country,2016,2015,2014,2013,2012,2011,2010
0,0,Albania,5.07,4.77,4.81,5.06,5.43,5.65,5.53
5,5,Algeria,0.56,0.56,0.56,0.54,0.49,0.44,0.39
10,10,Andorra,10.06,9.97,9.95,9.78,10.06,10.31,10.64
15,15,Angola,4.70,5.65,9.00,8.02,8.14,7.86,7.67
20,20,Armenia,3.77,3.84,3.97,3.72,3.84,4.06,4.23


In [152]:
apc_data.to_csv('data/apc_data.csv', index=False)

## Tobac ad cleaning

In [ ]:
# tobac_ad_raw.rename(columns={})

In [ ]:
# tobac_ad_ban = tobac_ad_raw.rename(columns={})

## Country data

In [158]:
gdp_cols = ['Country', 'Population', 'GDP ($ per capita)']
gdp_func_cols = [country_str_strip]

In [174]:
gdp_data = generic_clean(df_raw=country_data,
                         columns=gdp_cols,
                         apply_func_cols=gdp_func_cols,
                         save_path="data/gdp_data.csv")

In [176]:
gdp_data.head()

,index,Country,Population,GDP ($ per capita)
0,0,Afghanistan,31056997,700.0
1,1,Albania,3581655,4500.0
2,2,Algeria,32930091,6000.0
3,3,American Samoa,57794,8000.0
4,4,Andorra,71201,19000.0


## Merging different datasets based on country
    
    The name of countries are not the same over different datasets.
    
    - Preparing a list of countries with same names
    


### Unifiying countries name

In [177]:
gdp_map_countries = {"Bolivia": "Bolivia (Plurinational State of)", 
                     "Bosnia & Herzegovina":"Bosnia and Herzegovina",
                    "Brunei": "Brunei Darussalam",
                    "Cape Verde": "Cabo Verde",
                    "Czech Republic": "Czechia",
                    "Iran": "Iran (Islamic Republic of)",
                    "United Kingdom": "United Kingdom of Great Britain and Northern Ireland",
                    "United States": "United States of America",
                    "Vietnam": "Viet Nam"}

cost_map_countries = {"Democratic Republic of the Congo":"Congo",}

In [178]:
tobac_cost = tobac_cost.replace( {"Democratic Republic of the Congo":"Congo"})
gdp_data = gdp_data.replace(gdp_map_countries)
# tobac_cost[tobac_cost['Country']=="Democratic Republic of the Congo"]

In [179]:
cost_countries = tobac_cost['Country'].unique()
use_countries = tobac_use['Country'].unique()
apc_countries = apc_data['Country'].unique()
gdp_countries = gdp_data['Country'].unique()
tax_countries = tobac_tax['Country'].unique()
ppp_countries = ppp_data['Country'].unique()
print(f"Number of countries in tobacco use table: {len(use_countries)}")
print(f"Number of countries in tobacco cost table: {len(cost_countries)}")
print(f"Number of countries in tobacco tax table: {len(tax_countries)}")
print(f"Number of countries in APC table: {len(apc_countries)}")
print(f"Number of countries in GDP table: {len(gdp_countries)}")
print(f"Number of countries in PPP table: {len(ppp_countries)}")

Number of countries in tobacco use table: 127
Number of countries in tobacco cost table: 172
Number of countries in tobacco tax table: 194
Number of countries in APC table: 93
Number of countries in GDP table: 226
Number of countries in PPP table: 57


In [181]:
same_countries = reduce(np.intersect1d, (cost_countries, use_countries, apc_countries, gdp_countries, tax_countries)) 
print(f"Number of countries with same names: {len(same_countries)}")

Number of countries with same names: 67


In [182]:
same_countries

array(['Albania', 'Armenia', 'Australia', 'Azerbaijan', 'Bahrain',
       'Belarus', 'Bolivia (Plurinational State of)',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cambodia',
       'Canada', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Czechia',
       'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Estonia',
       'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Honduras',
       'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Jordan', 'Kazakhstan', 'Kyrgyzstan', 'Latvia',
       'Lebanon', 'Lithuania', 'Luxembourg', 'Malaysia', 'Malta',
       'Mexico', 'Mozambique', 'Norway', 'Oman', 'Pakistan', 'Panama',
       'Paraguay', 'Philippines', 'Poland', 'Portugal', 'Serbia',
       'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 'Spain',
       'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
       'United Kingdom of Great Britain and Northern Ireland',
       'Uzbekistan', 'Viet Nam'], dtype=object)

In [183]:
cost_diff = cost_countries[np.isin(cost_countries, same_countries, invert=True)]
use_diff = use_countries[np.isin(use_countries, same_countries, invert=True)]
apc_diff = apc_countries[np.isin(apc_countries, same_countries, invert=True)]
gdp_diff = gdp_countries[np.isin(gdp_countries, same_countries, invert=True)]

In [184]:
# map_countries = {   "Bolivia (Plurinational State of)": "Bolivia", 
#                     "Brunei Darussalam":"Brunei",
#                     "Iran (Islamic Republic of)":"Iran",
# #                     "United Kingdom": "United Kingdom of Great Britain and Northern Ireland",
#                     "United States of America":"United States",
#                     "Viet Nam":"Vietnam"}

In [187]:
tobac_use = tobac_use[ tobac_use['Country'].isin( same_countries)]
tobac_cost = tobac_cost[ tobac_cost['Country'].isin( same_countries)]
tobac_tax = tobac_tax[tobac_tax['Country'].isin(same_countries)]
apc_data = apc_data[ apc_data['Country'].isin( same_countries)]
gdp_data = gdp_data[ gdp_data['Country'].isin( same_countries)]

### Selecting rows with based on available years

In [188]:
use_year = tobac_use['Year'].unique()
cost_year = tobac_cost['Year'].unique()
tax_year = tobac_tax['Year'].unique()
apc_year = ['2016', '2015', '2014', '2013', '2012', '2011', '2010']

In [ ]:
use_year

In [ ]:
cost_year

In [ ]:
tax_year

In [ ]:
# gdp_tobac_use = pd.merge(gdp_data, tobac_use, on='Country') 

In [ ]:
# gdp_tobac_use = gdp_tobac_use.rename(columns={"Male": "Male Tobacco Use%", "Female": "Female Tobacco Use%"})

In [ ]:
# gdp_tobac_use[gdp_tobac_use['Country']=='Germany'] 

In [ ]:
# gdp_tobac_use.plot.scatter(x='GDP ($ per capita)',
#                            y='Male Tobacco Use%')

In [ ]:
# Make csv of clean data.
# tobac_use.to_csv('data/Clean_Data/cl-tobac-use.csv')
# tobac_cost.to_csv('data/Clean_Data/cl-tobac-cost.csv')
# gdp_data.to_csv('data/Clean_Data/cl-gdp-data.csv')